<a href="https://colab.research.google.com/github/e19166/e19-4yp-Dynamic-Multi-Dimensional-Resource-Orchestration-in-Kubernetes/blob/main/Models/RLS/Recursive_least_square.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Service 1

In [1]:
!pip install river optuna pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 133.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 wh

In [3]:
import pandas as pd
import numpy as np
from river import linear_model, metrics, preprocessing, compose
import optuna

# Load data
df = pd.read_csv("/content/service-1-deployment_dataset.csv")

# Timestamp processing
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format = 'mixed')
df = df.sort_values("Timestamp")

# Feature engineering
df['cpu_usage_pct'] = df['CPU Usage'] / df['CPU Limit']
df['memory_usage_pct'] = df['Memory Usage'] / df['Memory Limit']
df['request_rate_rps'] = df['Request Rate']
df['cpu_allocated'] = df['CPU Request']
df['memory_allocated'] = df['Memory Request']
df['hour'] = df['Timestamp'].dt.hour
df['dayofweek'] = df['Timestamp'].dt.dayofweek

# Rolling usage trends
df['cpu_usage_mean_5'] = df['cpu_usage_pct'].rolling(window=5).mean().bfill()
df['memory_usage_mean_5'] = df['memory_usage_pct'].rolling(window=5).mean().bfill()
df['cpu_usage_std_5'] = df['cpu_usage_pct'].rolling(window=5).std().bfill()
df['memory_usage_std_5'] = df['memory_usage_pct'].rolling(window=5).std().bfill()

# Targets (next-step CPU & Memory usage)
df['cpu_needed_t+1'] = df['CPU Usage'].shift(-1)
df['memory_needed_t+1'] = df['Memory Usage'].shift(-1)

# Drop last row with NaNs
df = df.dropna()


In [4]:
features = [
    'cpu_usage_pct', 'memory_usage_pct', 'request_rate_rps',
    'cpu_allocated', 'memory_allocated',
    'cpu_usage_mean_5', 'memory_usage_mean_5',
    'cpu_usage_std_5', 'memory_usage_std_5',
    'hour', 'dayofweek'
]

X = df[features].to_dict(orient='records')
y_cpu = df['cpu_needed_t+1'].values
y_mem = df['memory_needed_t+1'].values


In [5]:
def evaluate_rls(params):
    cpu_model = compose.Pipeline(
        preprocessing.StandardScaler(),
        linear_model.RecursiveLeastSquares(
            alpha=params['alpha'],
            initializer=params['initializer']
        )
    )

    mem_model = compose.Pipeline(
        preprocessing.StandardScaler(),
        linear_model.RecursiveLeastSquares(
            alpha=params['alpha'],
            initializer=params['initializer']
        )
    )

    mae_cpu = metrics.MAE()
    r2_cpu = metrics.R2()

    mae_mem = metrics.MAE()
    r2_mem = metrics.R2()

    for xi, y_cpu_i, y_mem_i in zip(X, y_cpu, y_mem):
        # Predict
        y_cpu_pred = cpu_model.predict_one(xi)
        y_mem_pred = mem_model.predict_one(xi)

        # Update metrics
        if y_cpu_pred is not None:
            mae_cpu.update(y_cpu_i, y_cpu_pred)
            r2_cpu.update(y_cpu_i, y_cpu_pred)
        if y_mem_pred is not None:
            mae_mem.update(y_mem_i, y_mem_pred)
            r2_mem.update(y_mem_i, y_mem_pred)

        # Learn
        cpu_model.learn_one(xi, y_cpu_i)
        mem_model.learn_one(xi, y_mem_i)

    return mae_cpu.get(), r2_cpu.get(), mae_mem.get(), r2_mem.get()
